<a href="https://colab.research.google.com/github/MarioPulpo95/DMDAS/blob/main/DenseDepthNewDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Nunzio03/DenseDepth_eval01

In [ ]:
!wget https://s3-eu-west-1.amazonaws.com/densedepth/kitti.h5 -O ./DenseDepth_eval01/kitti.h5

In [ ]:
import os
import glob
import argparse
import matplotlib

# Keras / TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'
from keras.models import load_model
from DenseDepth_eval01.layers import BilinearUpSampling2D
from tensorflow.keras.layers import Layer, InputSpec
from DenseDepth_eval01.utils import predict, load_images, display_images
from matplotlib import pyplot as plt
from DenseDepth_eval01.validation_metrics import mean_absolute_error, root_mean_squared_error, relative_absolute_error, relative_squared_error, delta1, f1_score, precision_recall, accuracy, completeness_accuracy,ssim, intersection_over_union, normalized_median_error, gradient_error, chamfer_distance, hausdorff_distance, mean_squared_error
from scipy.ndimage import zoom
import numpy as np
import cv2

'''# Argument Parser
parser = argparse.ArgumentParser(description='High Quality Monocular Depth Estimation via Transfer Learning')
#parser.add_argument('--model', default='nyu.h5', type=str, help='Trained Keras model file.')
parser.add_argument('--model', default='kitti.h5', type=str, help='Trained Keras model file.')
#parser.add_argument('--input', default='examples/*.png', type=str, help='Input filename or folder.')
parser.add_argument('--input', default='miniset/*.png', type=str, help='Input filename or folder.')
parser.add_argument('--gtinputs', default='miniset/groundtruth/*.png', type=str, help='Input filename or folder.')
args = parser.parse_args()'''

# Custom object needed for inference and training
custom_objects = {'BilinearUpSampling2D': BilinearUpSampling2D, 'depth_loss_function': None}

print('Loading model...')

# Load model into GPU / CPU
model = load_model('DenseDepth_eval01/kitti.h5', custom_objects=custom_objects, compile=False)

print('\nModel loaded ({0}).'.format('DenseDepth_eval01/kitti.h5'))

# Input images
inputs = load_images( glob.glob('DenseDepth_eval01/miniset/*.png') )
print('\nLoaded ({0}) images of size {1}.'.format(inputs.shape[0], inputs.shape[1:]))

# Compute results
outputs = predict(model, inputs)
#print('Outputs:{}'.format(outputs))

gtinputs = load_images(glob.glob('DenseDepth_eval01/miniset/groundtruth/*.png'))

#matplotlib problem on ubuntu terminal fix
#matplotlib.use('TkAgg')   

# zomm output images to compute metrics because inputs and outputs have different dimensions
outputs = zoom(outputs, (1, 2, 2, 1))
#gtinputs = np.expand_dims(gtinputs, axis=3)
#gtinputs = np.repeat(gtinputs, 3, axis=3)
outputs = np.squeeze(outputs, axis=-1)

print(outputs.shape)  # predizione  # 1 canale
print(gtinputs.shape)  # groundtruth # 1 canali 

#print(outputs)  # predizione  # 1 canale
#print(gtinputs)  # groundtruth # 1 canali 

# Controllo che output e groundtruth abbiano la stessa scala
#print('outputs type:{}'.format(type(outputs)))  # predizione
#print('gtinputs type:{}'.format(type(gtinputs))) # groundtruth

In [ ]:
plt.imshow(outputs[0], cmap='hot')
plt.show()
print(outputs[0])

In [ ]:
plt.imshow(gtinputs[0], cmap='hot')
plt.show()
print(gtinputs[0])

In [ ]:
for img_out, img_gt in zip(outputs, gtinputs):
  MAE = mean_absolute_error(img_out, img_gt)
  MSE = mean_squared_error(img_out, img_gt)
  SSIM = ssim(img_out, img_gt)
  RMSE = root_mean_squared_error(img_out, img_gt)
  RAE = relative_absolute_error(img_out, img_gt)
  RSE = relative_squared_error(img_out, img_gt)
  GRADIENT = gradient_error(img_out, img_gt)
  NME = normalized_median_error(img_out, img_gt)
  ACCURACY = accuracy(img_out, img_gt)
  print('Mean Absolute Error:', MAE)
  print('Structural Similarity Index:', SSIM)
  print('Mean Squared Error:', MSE)
  print('Relative Absolute Error:', RAE)
  print('Relative Squared Error:', RSE)
  print('Root Mean Squared Error:', RMSE)
  print('Normalized Median Error:', NME)
  print('Gradient:', GRADIENT)